# 1.2.2: Bikeshare (Implementing the State)

<br>



---



*Modeling and Simulation in Python*


Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---





We have now transformed a system into a model by defining the 4 types of variables.  Let's review our decisions:

<br>

Parameters:
* Total number of bikes: 12
* Length of simulation: 15 hours
* Time steps: 15 minutes (for a total of 60 time steps)
* Chance of bike being picked up in a time step: 50% (from Augie), 40% (from Moline)

<br>

Independent variables:
* Number of bikes starting at Augie: 0-12 (the initial state)

<br>

State variables:
* Number of bikes at Augie and Moline at a given moment

<br>

Metrics:
* Chance of having no empty racks?  Number of unhappy customers?

<br>

Now we need to begin the implementation process. We define the parameters and independent variables before the simulation begins, and metrics won't become important until we get a running simulation.  So our main aim will be to establish:
* a tool to keep track of the state variables as they change.
* a rule for how they will change (*a change function*).

Let's get to work!
<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Simulatin.PNG width = 400>
</center>

## Keeping track of the state variables

In order to store the state, we'll use an *object* called a `Series`, which is part of a data analysis library called Pandas.  A couple definitions:

- An `object` is a catch-all term for a "thing" in python: a data storage vehicle of any sort.

- A `Series` is an object that holds a set of variable values in a table-like form, like a two-column spreadsheet.  We'll see an example of this in action in just a moment.


We'll start by importing the Pandas library (just as we have import Numpy in previous notebooks), and creating a `Series` called `bikeshare`.  This will be our *state object*: it will hold up-to-date information about the state of our system.

In [1]:
import pandas as pd
state = dict(augie=10,moline=2)
bikeshare = pd.Series(state,name='Number of Bikes')
bikeshare.index.name = 'Location'
bikeshare

,Number of Bikes
Location,
augie,10
moline,2


What did we just do?

* The first line imports the library, and gives it the nickname `pd`.  Importing the library makes all of its objects and methods available to the program, and the shorthand version slims down the code a bit.

<br>

* The second line creates a foundational Python object called a `dictionary`, which is a set of label-value pairs (think of it like a word and a definition--thus the name dictionary!).  The two labels in the dictionary ('Augie' and 'Moline') represent our *state variables*.  The *initial state* indicates that there are 10 bikes at Augie and 2 at Moline.

<br>

* The third line creates the `Series` and *assigns* it to a variable name `bikeshare`.  `bikeshare` now refers to the Series, in the same way that `g` refers to `9.8` when we write `g = 9.8`. A `Series` is a lot like a dictionary, but it is much easier to manipulate and is a more powerful way to store data.  The `name` is a *keyword argument* that describes the meaning of our values: '10' and '2' represent the number of bikes at the two locations.

<br>  

* The fourth line gives the *index* a name.  The index is the list of labels: the left column of the two-column table.  Here the index is the list of places with a bike rack.

<br>

* The final line sends the contents of `bikeshare` to be output, which is why the details of the Series appear when we run the cell.

We can read the variables inside a `Series` using the *dot operator*, like this:

In [2]:
bikeshare.augie

10

In [3]:
bikeshare.moline

2

Or, to display the state variables and their values, you can just enter the name of the object:

In [4]:
bikeshare

,Number of Bikes
Location,
augie,10
moline,2


We'll use Series a lot, for a range of purposes, so take a minute to understand this code.  Just remember that a Series is simply a two-column table, with an "index" of labels and a column of values.  Like this:

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Series.PNG width = 300>
</center>

## The change function

Ok, we now have a place to store the state variables.  But any useful model is going to involve change.

<br>

In a model, the process of change must be defined by a rule, or a set of rules, about how this change occurs.  We will call these rules a *change function*.  Much of this class will be spent exploring different types of change functions.  These rules might stay the same the whole time, or they might change, gradually or suddenly.  The rules might create random *stochastic* changes, or established *deterministic* changes.

<br>

The change occurs through *time steps*.  Each time step represents a certain amount of time, and each step represents a change in the state of the system.  We have defined our time step as being 15 minutes.

### Changing the state manually

At its most basic, we enact the change function manually: that is, we can update the state by assigning new values to the variables.
For example, if a student moves a bike from Augie to Moline, we can figure out the new values and assign them:

In [5]:
# First simple change function
bikeshare.augie = 9
bikeshare.moline = 3
bikeshare

,Number of Bikes
Location,
augie,9
moline,3


That's not particularly efficient, though.  One step better, we can avoid doing the math ourselves by using *update operators*, `-=` and `+=`, to subtract 1 from
`augie` and add 1 to `moline`:

In [6]:
# Second simple change function
bikeshare.augie -= 1
bikeshare.moline += 1
bikeshare

,Number of Bikes
Location,
augie,8
moline,4


Try running the last cell again.  What happens to the state of the system?  Now run the previous code cell (with the title "first simple change function") again.  Finally, run the last cell again.

<br>

Each time you run the cell, it performs the action described by the code, even if you had already run that cell before.  This can be useful, but it can also cause trouble if you are not paying close attention!

### Introducing the idea of a coding function

The advantage of using a computer to create a model (i.e. of creating *simulations*) is that we don't have to manually enter the change for each time step.  We want to define the rule (that is, the change function) and let the computer do the grunt work.

<br>

When we want to do something over and over in code, we use a *function*.  A  function allows us to write a few lines of code, test them to confirm they do what we intend, and then *call* that function everytime we want to perform those actions.

<br>

What is a function?  It is simply a tool that takes an input and produces an output.  Think about how the term is used in math.  A mathematical function, like $y = 6x +4$, takes an input $x$ and produces an output $y$.  Sometimes, mathematicians say, "y is a function of x", and write that as:

<br>

$$y=f(x)$$

<br>

A physicist, on the other hand, might observe that for a penny falling from the Empire state building, "the time of falling $t$ a function of the orientation $O$ of the penny as well as the density of the air $\rho$", and write:

<br>

$$t = f(O,\rho)$$

<br>

Notice that when we write it this way, we can't see what the function *does*: we know that the value $y$ depends on $x$ and that the value $t$ depends on $\rho$, but we're not sure how.  This is often true with coding functions.  Consider a function we've already used, `np.sin()`:



In [8]:
import numpy as np
x = 0.5
y = np.sin(0.5)
print(y)

0.479425538604203


We don't know how `np.sin` converts the input $x$ into the output $y$, and oftentimes we don't particularly care.  Notice there is a second function in the cell above: `print` takes the variable $y$ is the input, and outputs the printed version below the cell.

<br>

Remember: just like a mathematical function, a coding function takes an input inside of parentheses and produces an output!

### Using a function to simplify our code

We can use functions that other coders have created, like `np.sin()`, `print()`, and `round()`.   But we can also create our own!  For example, these lines move a bike from Augie to Moline:

In [7]:
# Not inside a function
bikeshare.augie -= 1
bikeshare.moline += 1

Rather than repeat them every time a bike moves, we can define a new
function.

In [ ]:
# Inside a function
def bike_to_moline():
    bikeshare.augie -= 1
    bikeshare.moline += 1
    return

`def` is a special word in Python that indicates we are defining a new
function: a set of instructions. The name of the function is `bike_to_moline`. Notice some of the details:

<br>

* The empty
parentheses indicate that this function requires no additional
information when it runs.
* The colon indicates the beginning of an
indented *code block*.
* The next three lines are the *body* of the function. They *have* to be indented; by convention, the indentation is four spaces.  Always follow this convention!
* `return` ends the function.
* Notice the name of the function: always choose a name that describes what a function does, as this makes reading and editing the code much easier.

<br>

When you define a function, it has no immediate effect, because all you have done is to create a set of instructions (i.e. the function).  

<br>

Open the variable window `{x}` and run the "Inside a function" cell repeatedly.  Confirm that the values of `bikeshare` do not change.  Now run the cell "Not inside a function" a couple times.  Check to see that the values of `bikeshare` change.  *Defining a function is not the same as calling the function!*

The "instructions" in the function are not followed until you *call* the function. Here's how to call
this function.  Notice there is no colon or `def`:

In [ ]:
bike_to_moline()
bikeshare

Location
augie     5
moline    7
Name: Number of Bikes, dtype: int64

When you call the function, it runs the steps in the body of the function, which
update the variables of the `bikeshare` object.  Notice that now the state variables are changing.

When you call a function, you have to include the parentheses. If you
leave them out, you get this:

In [ ]:
bike_to_moline

<function __main__.bike_to_moline()>

This result indicates that `bike_to_moline` is a function. You don't have to know what `__main__` means, but if you see something like this, it probably means that you named a function but didn't actually call it.
So don't forget the parentheses.

<br>

The chief benefit of defining functions is that you avoid repeating chunks of
code, which makes programs smaller and much easier to read, edit, and debug.




### Monitoring the state with `print()`

As you write more complicated programs, it is easy to lose track of what
is going on. One of the most useful tools for debugging is the *print statement*, which displays text in cell output.

<br>

Normally when Colab runs the code in a cell, it displays the value of
the last line of code. For example, if you run:

In [ ]:
bikeshare.augie
bikeshare.moline

10

Colab runs both lines, but it only displays the value of the
second. If you want to display more than one value, you can use
print statements:

In [ ]:
print(bikeshare.augie)
print(bikeshare.moline)

2
10


When you call the `print` function, you can print a single data object or you can provide a sequence separated by commas, like this:

In [ ]:
print("There are", bikeshare.augie,"bikes at Augustana, and",
      bikeshare.moline, "bikes in Moline.")

There are 2 bikes at Augustana, and 10 bikes in Moline.


Notice that when you put letters or numbers inside quotation marks, the letters/ numbers are not treated as variables or numerical values.  These are called `strings`.

<br>

Print statements are useful for monitoring changing variables and debugging functions. For example, we can
add a print statement to `bike_to_moline`, like this:

In [ ]:
def bike_to_moline():
    print('Before Moving a bike to Moline', bikeshare.moline)
    bikeshare.augie += 1
    bikeshare.moline -= 1
    print('After moving a bike to Moline', bikeshare.moline)

Each time we call this version of the function, it displays a message,
which can help us keep track of what the program is doing.
 If I were to call this function, and the value of `bikeshare.moline` did not change, or became smaller, I would know that something was off with the code in between the two print statements.


In [ ]:
bike_to_moline()
bikeshare

Before Moving a bike to Moline 5
After moving a bike to Moline 4


Location
augie     8
moline    4
Name: Number of Bikes, dtype: int64

Oops!  Looks like our code is taking a bike away from moline instead of adding one.  Go above and fix the function to be correctly add and subtract the bikes!

<br>



---

## Summary and Exercises

This chapter introduces the tools we need to keep track of the state of a system and to change that state.   In the next chapter, we'll use these tools to show the change in the system over the course of a day.



---
<br>

🟨 🟨

### Exercise 1

Make a state Series, but this time add a third state variable in addition to `augie` and `moline`.  Call the third variable `rock_island`, with initial value 0, and display the state of the system.

In [ ]:
# Solution goes here


---
<br>

🟨 🟨

### Exercise 2

Create a new Series that contains the information shown in this two column table and output it:

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Series_ex.PNG width = 300>
</center>

In [ ]:
# Solution goes here

---
<br>

🟨 🟨

### Exercise 3

Write a `bike_to_augie()` function that changes the state variables appropriately and then prints out the number of bikes at Augie after the change.

<br>

Call the function to make sure it works.

In [ ]:
# Function goes here
def bike_to_augie():


In [ ]:
# Run this cell to call the function here
state = dict(augie=10,moline=2)
bikeshare = pd.Series(state,name='Number of Bikes')
bike_to_augie()


d
